In [1]:
import itertools as it
import numpy as np
import functools as ft
import operator
from numpy.linalg import norm
import sympy as sp
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, transpile
from qiskit.quantum_info import Statevector, random_statevector
from qiskit.quantum_info import state_fidelity as fid
from tqdm import tqdm
import qutip as qt
import time

In [2]:
def Dec2Bin(x, n):
    return np.array([int(i) for i in np.binary_repr(x, width=n)])

In [3]:
def get_proj(n):
    circ_idx = {}    
    T_Proj = []

    states = ['0','1']
    
    idx_r = [0,1]
 
        
    for j in range(n):
        z_len = n-(j+1)
        b_len = 2**(z_len) - 1
        m_len = (j+1)-1
  
        if b_len!=0:
            for b in range(b_len+1):
                proj = []
                Bin = Dec2Bin(b, z_len)
                for i in Bin:
                    proj.append(states[i])
                proj.append(states[idx_r[0]])
                        
                for i in range(m_len):
                        
                    proj.append(states[idx_r[1]])
                        
                T_Proj.append(''.join(proj))
        else:
            proj = []
            proj.append(states[idx_r[0]])
            for i in range(m_len):
                        proj.append(states[idx_r[1]])
            
            T_Proj.append(''.join(proj))
        circ_idx[j] = T_Proj
        T_Proj = []
        
        
    return  circ_idx

In [4]:
def get_keys(n):
    keys = []
    for i in range(2**n):
        keys.append(format(i, '0'+str(n)+'b'))
    return keys

In [5]:
def IBM_counts(n,PSI,a,shots,qcomp):
    proj  = get_proj(n)
    prob_dict = {}
    for i in range(n):
        qc = QuantumCircuit(n)
        #prepare n qubit randoom state
        qc.initialize(PSI, range(n))
        for j in range(i+1):
            if a == 1:
                qc.h(i-j)
            else:
                qc.sdg(i-j)
                qc.h(i-j)
        qc.measure_all()
        job = execute(transpile(qc, qcomp), qcomp, shots=shots)
        result = job.result()
        counts = result.get_counts(qc)
       
        values = list( map(counts.get, proj[i],[0]*len(proj[i])) )
     
           
        prob_dict[i] = np.array(values)/shots
        
 
        
    return prob_dict

In [6]:
def comp_IBM(n,PSI,shots,qcomp):
    qc = QuantumCircuit(n)
    #prepare n qubit randoom state
    qc.initialize(PSI, range(n))
    qc.measure_all()
    job = execute(transpile(qc, qcomp), qcomp, shots=shots)
    result = job.result()
    counts = result.get_counts(qc)
    keys = get_keys(n)
    probs = np.array(list( map(counts.get, keys,[0]*len(keys)) ))/shots
    return np.sqrt(probs)

In [7]:
def scalable(prob_comp,prob_dict_1,prob_dict_2,n):
    ket_0 = np.array([1,0])
    ket_1 = np.array([0,1])
    minus = np.array([1,-1])/np.sqrt(2)
    minusj = np.array([1,-1j])/np.sqrt(2)

    st_temp = []
    ct = 0
    j=0
    for i in range(len(prob_dict_1[j])):
        P_1_hat_1=(1/(prob_comp[ct]*prob_comp[ct+1]))*(prob_dict_1[j][i]-(1/2)*((prob_comp[ct])**2+(prob_comp[ct+1])**2))
        P_1_hat_2=(1/(prob_comp[ct]*prob_comp[ct+1]))*(prob_dict_2[j][i]-(1/2)*((prob_comp[ct])**2+(prob_comp[ct+1])**2))
        phase = P_1_hat_1 + 1j*P_1_hat_2
        st = prob_comp[ct]*ket_0 + phase*prob_comp[ct+1]*ket_1
        st_temp.append(st)
        ct+=2
    for j in range(1,n):       
        W_0 = ft.reduce(np.kron,[minus]*(j))
        W_1 = ft.reduce(np.kron,[minusj]*(j))
        st_temp_2 = []
        for beta in range(0,2**(n-j-1)):
            ind_0 = 2*beta
            ind_1 = 2*beta+1
        # print(beta)
            X_0=np.dot(st_temp[ind_0].conj(),W_0) *np.dot(W_0.conj(),st_temp[ind_1])
            X_1=np.exp(-1j*np.pi/2)*np.dot(st_temp[ind_0].conj(),W_1) *np.dot(W_1.conj(),st_temp[ind_1])
            P_10_hat_2=prob_dict_1[j][beta]-(1/2)*abs(np.dot(W_0.conj(),st_temp[ind_0]))**2 - (1/2)*abs(np.dot(W_0.conj(),st_temp[ind_1]))**2
            P_11_hat_2=prob_dict_2[j][beta]-(1/2)*abs(np.dot(W_1.conj(),st_temp[ind_0]))**2 - (1/2)*abs(np.dot(W_1.conj(),st_temp[ind_1]))**2
            M=np.array([[np.real(X_0),-np.imag(X_0)],[np.real(X_1),-np.imag(X_1)]])
            P=np.array([P_10_hat_2,P_11_hat_2])
            X=np.linalg.solve(M,P)
            Val=X[0]+1j*X[1]
            st = np.kron(ket_0,st_temp[ind_0]) + Val*np.kron(ket_1,st_temp[ind_1])
            st_temp_2.append(st)
        st_temp = st_temp_2
    state = st/norm(st)
    return state

In [8]:
qcomp = Aer.get_backend('qasm_simulator')

In [ ]:
n = 2
d = 2**n
avg=100 # number of averages
Fid = []
Time = []
avg_fid = []
avg_time = []
cop = []
po = 5
for c in range(po):
    shots = 100000 * (10**c) # number of copies
    for k in tqdm(range(avg)):
     ########### Config ###########
        
        
        psi=np.array(qt.rand_ket_haar(d)) # random state
        #psi = np.load(str(n)+'_psi.npy')
        #psi = qt.ghz_state(n).full()
        #psi = np.array([-0.00827484+0.54549734*1j,  0.0030454 -0.12905376*1j, 0.64517208-0.27475179*1j,  0.14379161-0.41627929*1j])
        rho=np.outer(psi,psi.conj())
        
        PSI=Statevector(psi)
        a=1
        st_sca = time.time()
        prob_dict_1 = IBM_counts(n,PSI,a,shots,qcomp)
        a=2
        prob_dict_2 = IBM_counts(n,PSI,a,shots,qcomp)
        prob_comp = comp_IBM(n,PSI,shots,qcomp)
        rho_est = scalable(prob_comp,prob_dict_1,prob_dict_2,n)
        end_sca = time.time()
        Fid.append(1-fid(psi,rho_est))
        Time.append(end_sca-st_sca)
    avg_fid.append(np.median(Fid))
    avg_time.append(np.median(Time))
    print('Average Infidility', np.median(Fid))
    print('Average Time', np.median(Time))
    
    
cop.append(shots)
fid_dict = {'Copies':cop, 'SCA':np.array(avg_fid)}
time_dict = {'Copies':cop, 'SCA':np.array(avg_time)}

df_fid = pd.DataFrame(fid_dict)
df_time = pd.DataFrame(time_dict)
df_fid.to_csv(str(n)+'_infid_SCA.csv', index=False)
df_time.to_csv(str(n)+'_time_SCA.csv', index=False)  